In [3]:
import torch
import matplotlib.pyplot as plt
torch.cuda.is_available()

True

In [ ]:
class LRModel_ex(torch.nn):
    def __init__(self):
        self.w=torch.nn.Parameter(torch.randn(1,,dtype=torch.float16),requires_grad=True)
        self.b=torch.nn.Parameter(torch.randn(1,dtype=torch.float16),requires_grad=True)
    def forward(self, x:torch.Tensor)->torch.Tensor:
        return torch.nn.Linear()